# 2-D Slider

This is an example of how you could make your own two-dimensional slider widget.

Author: Tyler Murphy, Cal Poly Physics Major

## Python Code

First, the necessary libraries from IPython need to be imported. An excellent explanation of the basics is provided in parts one through five of the widgets tutorial found in ipython/examples/widgets

In [1]:
from __future__ import print_function
import IPython
from IPython.html import widgets
from IPython.display import display
from IPython.utils.traitlets import Unicode
from IPython.html.widgets import DOMWidget

ModuleNotFoundError: No module named 'IPython.html'

The following code declares which variables are synced between python and javascript using a backbone model. The values given are the defaults. Note that the first line imports the variable types that are used to describe the widget.

In [2]:
from IPython.utils.traitlets import Unicode, Int, Float

class TwoDimSlider(DOMWidget):
    _view_name = Unicode('TwoDimSliderView', sync=True)
    x = Float(0.0, sync=True)
    y = Float(0.0, sync=True)
    xmin = Float(0.0, sync=True)
    xmax = Float(1.0, sync=True)
    ymin = Float(0.0, sync=True)
    ymax = Float(1.0, sync=True)
    width = Int(100, sync=True)
    height = Int(100, sync=True)
    round_factor = Int(2, sync=True) #the number of digits after the decimal
    marker_size = Int(20, sync=True) #marker diameter in pixels

## Javascript

The following cell is defined as javascript using cell magics. The '%%javascript' must be on the first line of the cell and defines the remainder of the cell to contain javascript. Comments throughout the following cell explain what different parts of the code are for.

In [3]:
%%javascript

require(["widgets/js/widget"], function(WidgetManager) {
    
    //The 'TwoDimSliderView' below is the name of a particular view associated with this widget.
    //A widget can have more than one view, and each one would be defined in the same way, by
    //extending the DOMWidgetView. This widget happens to have just one view. The view displayed
    //is controlled just like any other parameter, using the _view_name key. 
    var TwoDimSliderView = IPython.DOMWidgetView.extend({
        
        //The view needs to have a function called 'render' which builds the slider.
        render: function() {
            
            // The parameters for the widget are retrieved using 'this.model.get('key_name')'
            // Get the bounds on the slider range
            var xmin = this.model.get('xmin');
            var ymin = this.model.get('ymin');
            var xmax = this.model.get('xmax');
            var ymax = this.model.get('ymax');
            
            // In general, don't store any state variables. Get them each time you need them so
            // they're not out of date. Within a function, though, it's okay to assign to a 
            // variable and reuse it rather than getting it several times. Getting it each time
            // is okay, too, since it's very fast.
            
            // Get the slider's size
            var x_size = this.model.get('width');
            var y_size = this.model.get('height');
            
            // Get marker size and position
            var marker_size = this.model.get('marker_size');
            var x = this.model.get('x');
            var y = this.model.get('y');
            
            // Create the slider elements:
            // The slider is built up piece by piece using different HTML elements appended to
            // each other. Jquery and css styling is used to control the look of each piece.
            
            //The markerbounds define the draggable area of the marker.
            this.$markerbounds = $('<div />')
                .appendTo(this.$el)
                .width(x_size + marker_size)
                .height(y_size + marker_size)
                .css({"margin-left": "15px"});
            
            // This box is a visual representation of the draggable area of the marker. It bounds
            // the center of the marker.
            this.$box = $('<div />')
                .appendTo(this.$markerbounds)
                .width(x_size)
                .height(y_size)
                .css({"top": marker_size / 2,
                      "left": marker_size / 2,
                      "position": "relative",
                     });
            
            this.$marker = $('<div />')
                .appendTo(this.$box)
                .width(marker_size)
                .height(marker_size)
                .css({"border-radius": 1000,
                      "background-color": "black",
                     });
            
            this.$coordx = $('<div class="coord" />')
                .appendTo(this.$el)
                .css({
                      "top": 0,
                      "position": "relative",
                      "text-align": "center",
                     });
            this.$xlabel = $('<div />')
                .appendTo(this.$coordx)
                .text('x:')
                .css({"display": "inline-block"});
            this.$xvalue = $('<div />')
                .appendTo(this.$coordx)
                .css({"display": "inline-block"});
            
            this.$coordy = $('<div class="coord" />')
                .appendTo(this.$el)
                .css({
                      "position": "relative",
                      "-webkit-transform": "rotate(-90deg)",
                      "-moz-transform": "rotate(-90deg)",
                      "-ms-transform": "rotate(-90deg)",
                      "-o-transform": "rotate(-90deg)",
                      "width": y_size,
                      "top": -y_size,
                      "left": -(y_size / 2) + 5,
                     });
            this.$ylabel = $('<div />')
                .appendTo(this.$coordy)
                .text('y: ')
                .css({"display": "inline-block"});
            this.$yvalue = $('<div />')
                .appendTo(this.$coordy)
                .css({"display": "inline-block"});
            
            // A Jquery UI function called draggable makes the marker draggable. Each time
            // the position of the marker changes the display_position and save_position
            // functions are called.
            var that = this;
            this.$marker.draggable({
                containment: this.$markerbounds,
                scroll: false,
                drag: function() {
                    that.display_position();
                    that.save_position();
                },
            });
            
            // Now, functions are defined to handle changes in the widget values that
            // necessitate a change in its appearance.
            // this.model.on("change:key_name", do_this()) is used to watch for changes
            // in a particular key and then execute the function do_this when there's
            // a change.
            
            //Handle slider position change:
            var handle_pos_change = function() {
                that.move_marker(that.model.get('x'),that.model.get('y'))
            };
            this.model.on("change:x", handle_pos_change);
            this.model.on("change:y", handle_pos_change);
            
            //Handle slider size change:
            //Width and height:
            var handle_size_change = function() {
                //Save the old x and y positions so that the marker can be moved and its value
                //from before resizing isn't lost:
                var x_old = that.model.get('x');
                var y_old = that.model.get('y');
                
                //Change the sizing and positioning of the relevant elements:
                that.$markerbounds.width(that.model.get('width') + that.model.get('marker_size'))
                                  .height(that.model.get('height') + that.model.get('marker_size'));
                that.$box.width(that.model.get('width'))
                        .height(that.model.get('height'))
                        .css({"top": that.model.get('marker_size') / 2,
                              "left": that.model.get('marker_size') / 2,
                              "position": "relative",
                         });
                that.$coordy.css({
                      "width": that.model.get('height'),
                      "top": -that.model.get('height'),
                      "left": -(that.model.get('height') / 2) + 5,
                     });
                that.$marker.width(that.model.get('marker_size'))
                            .height(that.model.get('marker_size'));
                
                //Move the marker to where it should be now that the size has changed.
                that.move_marker(x_old, y_old);
                that.get_position();
                that.display_position();
            };
            this.model.on("change:width", handle_size_change);
            this.model.on("change:height", handle_size_change);
            this.model.on("change:marker_size", handle_size_change);
            
            //Handle changes in the range of the slider:
            var handle_range_change = function() {
                that.get_position();
                that.save_position();
                that.display_position();
            };
            this.model.on("change:xmin", handle_range_change);
            this.model.on("change:xmax", handle_range_change);
            this.model.on("change:ymin", handle_range_change);
            this.model.on("change:ymax", handle_range_change);
                       
            //Move the marker to the correct starting position:
            this.move_marker(x, y);
            //And label the axes:
            this.$xvalue.html(x.toString());
            this.$yvalue.html(y.toString());
            
            // This line defines which element will be styled when using the set_css method
            // for the widget
            this.$el_to_style = this.$box;
            
        },
        
        //Other functions which do things like move the marker can also be defined.
        move_marker: function(x, y) {
            var y_size = this.model.get('height');
            var x_size = this.model.get('width');
            var marker_size = this.model.get('marker_size');
            
            var xmin = this.model.get('xmin');
            var xmax = this.model.get('xmax');
            var ymin = this.model.get('ymin');
            var ymax = this.model.get('ymax');
            
            if (x > xmax) {
                x = xmax;
            } else if (x < xmin) {
                x = xmin;
            }
            
            if (y > ymax) {
                y = ymax;
            } else if (y < ymin) {
                y = ymin;
            }
            
            //Calculate the necessary positions:
            var top = (y_size - (((y - ymin) / (ymax - ymin)) * y_size)) - (marker_size / 2);
            var left = (((x - xmin) / (xmax - xmin)) * x_size) - (marker_size / 2);
            
            //Move the marker
            this.$marker.css({
                "top": top.toString() + "px",
                "left": left.toString() + "px",
            });
            this.display_position();
        },
        
        get_round_factor: function() {
            var decimals = Math.pow(10,this.model.get('round_factor'));
            return decimals;
        },
        
        get_position: function() {         
            //Get values necessary to calculate position:
            var round_factor = this.get_round_factor();
            var xrange = [this.model.get('xmin'), this.model.get('xmax')];
            var yrange = [this.model.get('ymin'), this.model.get('ymax')];
            var width = this.model.get('width');
            var height = this.model.get('height');
            var marker_size = this.model.get('marker_size');
            
            //Get the slider's position on the page
            var marker_pos = this.$marker.position();
            var left_pos = marker_pos.left + marker_size / 2;
            var top_pos = marker_pos.top + marker_size / 2;
            
            //Calculate position
            var position = {
                x: Math.round(round_factor * ((left_pos * (xrange[1] - xrange[0]) / width) + xrange[0] )) / round_factor,
                y: Math.round(round_factor * (( (height - top_pos) * (yrange[1] - yrange[0]) / height) + yrange[0] )) / round_factor,
            };
            return position;
        },
        
        // The save_position function uses this.model.set to update values in the
        // backbone model. Call this.touch() after setting to push the values.
        save_position: function() {
            //Get position:
            var position = this.get_position();
            
            //Update the x- and y-values in python:
            this.model.set('x',position.x);
            this.model.set('y',position.y);
            this.touch();
        },
        
        display_position: function() {
            //Get position:
            var position = this.get_position();
            //Update the view:
            this.$xvalue.html(position.x.toString());
            this.$yvalue.html(position.y.toString());
        },

    });
    
    // Register the widget view with the widget manager using the python view name first, followed
    //by the name of the widget in javscript.
    WidgetManager.register_widget_view('TwoDimSliderView', TwoDimSliderView);
    
});

<IPython.core.display.Javascript object>

## Widget Usage

Use the following line to create an instance of the widget with the name 'w'.

In [4]:
w = TwoDimSlider()

Then, to display the widget, either call 'w', or just use the display command. The set_css method can be used to set css properties for whichever HTML element is assigned to this.$el_to_style in javascript. Note that the defaults defined in python were used to render the widget.

In [5]:
display(w)

In [6]:
w.set_css("background", "purple")

The position of the slider, the marker size, and the size of the slider can also be changed:

In [7]:
w.x = 0.5
w.y = 0.75

In [8]:
w.marker_size = 30

In [9]:
w.width = 150
w.height = 150

The range of the slider can be changed. Changing the range doesn't change the x- and y-values, so the position of the marker in the bounding box changes. The number of digits after the decimal the x- and y-values are rounded to is controlled using the round_factor parameter. Negative values for round_factor allow rounding to ones, tens, hundreds, etc.

In [10]:
w.xmin = -3
w.ymax = 3
w.round_factor = 1

The slider can be displayed again. Since it's just a visual representation of the data in the backbone model, every instance of the slider on the page moves together.

In [11]:
display(w)
display(w)

The slider can also be linked to other widgets. For example, the x-value of this slider is linked to a one-dimensional slider:

In [12]:
from IPython.utils.traitlets import link
x = widgets.FloatSliderWidget(min=-3.0, max=1.0, step=0.1)
link((w, 'x'), (x, 'value'))
display(x)

All of the keys for the widget can be viewed using the following command.

In [9]:
w.keys

['_view_name',
 'ymax',
 'msg_throttle',
 'width',
 'marker_size',
 '_css',
 'height',
 'visible',
 'xmax',
 'xmin',
 'x',
 'ymin',
 'y',
 'round_factor']

### Acknowledgments

Thanks to Brian Granger and Jonathan Frederic for all the help they gave me along the way.